In [1]:
import pandas as pd
import numpy as np
import gc
import timeit
from numpy.linalg import inv

In [2]:
start_time = timeit.default_timer()
df = pd.read_csv("/datasets/iowaliquor.csv")
df = df.iloc[::10, :]

elapsed = timeit.default_timer() - start_time
print("Time (minutes) elapsed for this cell:", elapsed/60)

df.head()

/opt/tljh/user/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Time (minutes) elapsed for this cell: 0.842103161966952


,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S08096000008,10/04/2012,4641,Kum & Go #573 / SE 14th DM,5830 SE 14th ST,DES MOINES,50315,NaN,77.0,Polk,...,89191,Jose Cuervo Especial Reposado Tequila Mini,12,500,11.50,17.25,1,17.25,0.5,0.13
10,S05372600066,05/03/2012,2527,Hy-Vee Food Store #5 / Des Moines,1107 SE ARMY POST RD,DES MOINES,50315,POINT (-93.602372 41.526289),77.0,Polk,...,43026,Admiral Nelson Spiced Rum,12,750,5.77,8.66,24,207.84,18.0,4.76
20,S07363200013,08/28/2012,2662,Hy-Vee Wine & Spirits / Muscatine,"522 MULBERRY, SUITE A",MUSCATINE,52761,POINT (-91.04437 41.426615),70.0,Muscatine,...,37418,Popov Vodka 80 Prf,6,1750,8.49,12.74,12,152.88,21.0,5.55
30,S18612600022,04/24/2014,3437,Scott's Foods,1120 SUNSET DRIVE,NORWALK,50211,POINT (-93.67803700000002 41.479944),91.0,Warren,...,54646,Arrow Blackberry Flav Brandy,12,750,5.38,8.06,12,96.72,9.0,2.38
40,S21013800004,09/03/2014,2596,Hy-Vee / Regal Liquors and Video,2453 NORTH COURT,OTTUMWA,52501,POINT (-92.408085 41.049854),90.0,Wapello,...,42141,UV Sugar Crush Mini,1,3000,29.30,43.95,1,43.95,3.0,0.79


In [3]:
df = df[["Date", "County", "Pack", "Category Name", "State Bottle Retail", "Volume Sold (Gallons)"]]

df.dropna(inplace=True)

category_name = df["Category Name"].copy()
category_name.loc[category_name.str.contains("WHISK")] = 'Whiskey'
category_name.loc[category_name.str.contains("VODKA")] = 'Vodka'
category_name.loc[category_name.str.contains("RUM")] = 'Rum'
category_name.loc[category_name.str.contains("SCHNAPPS")] = 'Schnapps'
category_name.loc[category_name.str.contains("BRAND")] = 'Brand'
category_name.loc[category_name.str.contains("GIN")] = 'Gin'
category_name.loc[category_name.str.contains("TEQUILA")] = 'Tequila'
category_name.loc[(category_name != 'Whiskey') &
                  (category_name != 'Vodka') &
                  (category_name != 'Rum') &
                  (category_name != 'Schapps') &
                  (category_name != 'Brand') &
                  (category_name != 'Gin') &
                  (category_name != 'Tequila')] = 'Other'

df["Category Name"] = category_name

df['County'] = df['County'].str.lower()
df.loc[df["County"]=="buena vist","County"] = "buena vista"
df.loc[df["County"]=="cerro gord","County"] = "cerro gordo"
df.loc[df["County"]=="obrien","County"] = "o'brien"
df.loc[df["County"]=="pottawatta","County"] = "pottawattamie"

# type casting
categorical_cols = ["Category Name"]
for col in categorical_cols:
    df[col] = df[col].astype('category')
    
# get year from date
df['Date'] = pd.to_datetime(df['Date'])
df.insert(1, 'Year', df['Date'].dt.year)


In [4]:
# AGGREGATION BY COUNTY AND YEAR
start_time = timeit.default_timer()

df.insert(1, 'Month-Year', df['Date'].dt.strftime('%m-%Y'))
df.drop(["Date", "Year"], axis=1, inplace=True)

df = df.groupby(['County', 'Month-Year']).agg('sum')

# add the year column back to the data frame
df.reset_index(level=["County", "Month-Year"], inplace=True)
df.insert(1, "Year", df["Month-Year"].str.replace("..-", ""))
df["Year"] = df["Year"].astype(int)

elapsed = timeit.default_timer() - start_time
print("Time (minutes) elapsed for this cell:", elapsed/60) 

Time (minutes) elapsed for this cell: 0.12804745081666624


In [5]:
# EXTERNAL DATASET, IOWA POPULATION FOR (county, year) PAIRS
df2 = pd.read_excel("iowa_county.xlsx")

# data cleaning/organization
df2 = df2.iloc[4:103]
df2.drop(["Unnamed: 1", "Unnamed: 2"], axis=1, inplace=True)
df2.rename(columns=
{"table with row headers in column A " +
 "and column headers in rows 3 through " +
 "4 (leading dots indicate sub-parts)": "County",
                    "Unnamed: 3": "2010", 
                    "Unnamed: 4": "2011", 
                    "Unnamed: 5": "2012", 
                    "Unnamed: 6": "2013", 
                    "Unnamed: 7": "2014",
                    "Unnamed: 8": "2015",
                    "Unnamed: 9": "2016",
                    "Unnamed: 10": "2017",
                    "Unnamed: 11": "2018",
                    "Unnamed: 12": "2019"}, inplace=True)
df2.reset_index(drop=True, inplace=True)
     
df2["County"] = df2["County"].str.replace(".", "")
df2["County"] = df2["County"].str.replace(" County, Iowa", "")
df2["County"] = df2['County'].str.lower()

# convert df to standard form
df2 = df2.melt(var_name="Year", value_name="Population", id_vars=['County'])

# type casting
df2["County"] = df2["County"].astype('category')
df2["Year"] = df2["Year"].astype('int')
df2["Population"] = df2["Population"].astype('int')

df2.head()

,County,Year,Population
0,adair,2010,7679
1,adams,2010,4023
2,allamakee,2010,14378
3,appanoose,2010,12856
4,audubon,2010,6098


In [6]:
# EXTERNAL DATASET, IOWA INCOME PER CAPITA FOR (county, year) PAIRS
df_income = pd.read_csv("income.csv")

df_income = df_income[["Name", "Variable", "Value", "Date"]]
df_income = df_income[df_income["Variable"] == "Per capita personal income"]


df_income.rename(columns={"Name": "County", "Value": "Income Per Capita"}, inplace=True)

df_income["County"] = df_income["County"].str.replace(", IA", "")
df_income["County"] = df_income["County"].str.lower()

df_income['Date'] = pd.to_datetime(df_income['Date'])
df_income["Year"] = df_income["Date"].dt.year

df_income.drop(["Variable", "Date"], axis=1, inplace=True)
df_income.head()

,County,Income Per Capita,Year
2073,adair,21269,1997
2074,adair,22305,1998
2075,adair,23030,1999
2076,adair,25831,2000
2077,adair,26276,2001


In [7]:
# EXTERNAL DATASET, IOWA 2016 ELECTION DATASET
df_election = pd.read_csv("iowa_election.csv", sep=';')
df_election = df_election[df_election["State"] == "Iowa"]
df_election.reset_index(inplace=True, drop=True)

df_election["County"] = df_election["County"].str.replace(" County, Iowa", "")
df_election["County"] = df_election["County"].str.lower()

cols_to_keep = ["County", "Precincts", "Votes", "Republicans 2016", "Democrats 2016", 
"Green 2016", "Libertarians 2016", "At Least High School Diploma",
"At Least Bachelors's Degree", "School Enrollment", "Median Earnings 2010",
"Children Under 6 Living in Poverty", "Adults 65 and Older Living in Poverty",
"Preschool.Enrollment.Ratio.enrolled.ages.3.and.4", "Poverty.Rate.below.federal.poverty.threshold",
"White", "Black", "Hispanic", "Asian", "Amerindian", "Other",
"Median Age", "Teen.births", "Sexually.transmitted.infections", "Unemployment", "Violent.crime"]

df_election = df_election[cols_to_keep]

df_election["Violent.crime"].fillna(df_election["Violent.crime"].mean(), inplace=True)
df_election["Sexually.transmitted.infections"].fillna(
    df_election["Sexually.transmitted.infections"].mean(), inplace=True)

df_election.head()

,County,Precincts,Votes,Republicans 2016,Democrats 2016,Green 2016,Libertarians 2016,At Least High School Diploma,At Least Bachelors's Degree,School Enrollment,...,Black,Hispanic,Asian,Amerindian,Other,Median Age,Teen.births,Sexually.transmitted.infections,Unemployment,Violent.crime
0,fayette,26.0,9813.0,56.975441,37.358606,0.631815,3.322124,88.2,16.5,82.10,...,0.85,1.80,0.50,0.10,0.80,42.4,28.1,252.7,0.057,185.69
1,madison,10.0,8523.0,62.607063,31.280066,0.481051,4.036138,93.0,18.2,80.25,...,0.30,1.45,0.15,0.10,0.90,39.3,26.1,107.7,0.059,54.11
2,palo alto,7.0,4674.0,65.875053,29.845956,0.427899,2.802739,87.2,16.1,76.50,...,0.25,1.35,0.15,0.15,0.45,42.4,21.5,159.3,0.041,145.71
3,buena vista,11.0,8179.0,59.848392,34.857562,0.574642,3.508986,79.5,22.6,80.75,...,1.95,21.50,4.80,0.10,1.20,36.9,44.7,225.4,0.045,275.61
4,marshall,20.0,17700.0,51.553672,43.067797,0.661017,3.254237,85.7,18.7,74.80,...,1.40,15.70,1.10,0.40,1.15,39.4,56.0,385.6,0.067,352.90


In [8]:
# MERGE ALL EXTERAL DATASETS
start_time = timeit.default_timer()

# merge with population values for each (county, year) pair
df = pd.merge(df, df2, on=["County", "Year"])

# per capita normalization
df["Volume Sold (Gallons) Per Capita"] = df["Volume Sold (Gallons)"]/df["Population"]

cols_to_drop = ["Volume Sold (Gallons)"]
df.drop(cols_to_drop, axis=1, inplace=True)

# merge with income values for each (county, year) pair
df = pd.merge(df, df_income, on=["County", "Year"])
df.drop(["Year"], axis=1, inplace=True)

# merge with election dataeset
df = pd.merge(df, df_election, on="County")

df.head()

,County,Month-Year,Pack,State Bottle Retail,Population,Volume Sold (Gallons) Per Capita,Income Per Capita,Precincts,Votes,Republicans 2016,...,Black,Hispanic,Asian,Amerindian,Other,Median Age,Teen.births,Sexually.transmitted.infections,Unemployment,Violent.crime
0,adair,01-2012,354,369.00,7468,0.009092,42093,6.0,3759.0,65.336526,...,0.05,0.95,0.25,0.1,0.65,44.8,26.4,146.1,0.042,48.19
1,adair,02-2012,395,524.48,7468,0.009487,42093,6.0,3759.0,65.336526,...,0.05,0.95,0.25,0.1,0.65,44.8,26.4,146.1,0.042,48.19
2,adair,03-2012,320,284.23,7468,0.005108,42093,6.0,3759.0,65.336526,...,0.05,0.95,0.25,0.1,0.65,44.8,26.4,146.1,0.042,48.19
3,adair,04-2012,510,458.76,7468,0.009859,42093,6.0,3759.0,65.336526,...,0.05,0.95,0.25,0.1,0.65,44.8,26.4,146.1,0.042,48.19
4,adair,05-2012,430,587.60,7468,0.010932,42093,6.0,3759.0,65.336526,...,0.05,0.95,0.25,0.1,0.65,44.8,26.4,146.1,0.042,48.19


In [9]:
iowa_month_county = df
iowa_month_county.to_csv("iowa_month_county.csv", index=False)